In [125]:
# import libraries
import pprint
import requests
import re
import pyjq
import json
from newspaper import Article
import pymongo
import datetime
from dateutil.rrule import rrule, MONTHLY
import math
import time

client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']
#clean this up for now
#coll.drop()

start = 1960
end = 2020
years = []
for i in range(end-start):
    years.append(start + i)

api_key = 'mXe9rrak6WGZA7H2zzeGejAbh0LoCAzY'

for i in range(len(years)):    
    url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=student+loan&fq=pub_year:("'+str(years[i])+'")&api-key=' + api_key
    r = requests .get(url)
    json_data = r.json()
    meta = pyjq.all('.response .meta',json_data)
    pages = math.ceil(meta[0]['hits']/10)
    time.sleep(20)
    for p in range(pages):
        url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=student+loan&'+str(p)+'fq=pub_year:("'+str(years[i])+'")&api-key=' + api_key
        r = requests .get(url)
        json_data = r.json()
        count = pyjq.all('.response .docs | length',json_data)[0]
        urls = pyjq.all('.response .docs[] .web_url',json_data)
        #years = pyjq.all('.response .docs[] .pub_year',json_data)

        for j in range(count):
            toi_article = Article(urls[j], language="en")
            try:
                toi_article.download() 
            except:
                continue
            try:
                toi_article.parse() 
            except:
                continue
            try:
                toi_article.nlp() 
            except:
                continue

            coll.insert_one ( {'title':toi_article.title,
                               'text': toi_article.text,
                               'summary': toi_article.summary, 
                               'keywords': toi_article.keywords, 
                               'year':years[i],
                               'url': urls[j] } )
        time.sleep(10)

In [137]:
client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']

print('Total NYT articles {0}'.format(str(coll.count_documents({}))))

Total NYT articles 19738


In [ ]:
client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']
pp = pprint.PrettyPrinter(indent=1,width=65)
for r in coll. find():
  print(pp .pformat (r))

In [195]:
import pandas as pd

client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']

boom=[]
genX=[]
genY=[]
genZ=[]

cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 1960, '$lte' : 1964 } } },{'$count': 'Article Count'}] )
boom = list(cursor)
cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 1965, '$lte' : 1979 } } },{'$count': 'Article Count'}] )
genX = list(cursor)
cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 1980, '$lte' : 1999 } } },{'$count': 'Article Count'}] )
genY = list(cursor)
cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 2000, '$lte' : 2019 } } },{'$count': 'Article Count'}] )
genZ = list(cursor)

df = pd.DataFrame(boom+genX+genY+genZ, index =['Boomers: 1960 to 1964',
                                               'Gen X:   1965 to 1979',
                                               'Gen Y:   1980 to 2000',
                                               'Gen Z:   2000 to 2019'],
                  columns =['Article Count']) 
display(df)
display(df.sum())

,Article Count
Boomers: 1960 to 1964,270
Gen X: 1965 to 1979,2810
Gen Y: 1980 to 2000,6890
Gen Z: 2000 to 2019,9768


Article Count    19738
dtype: int64